In [1]:
import cv2
import random
import os
import math
import numpy as np
import pydicom
import matplotlib.pyplot as plt

from PIL import Image
from skimage import color

import warnings
warnings.filterwarnings("ignore")

input_dir = '/kaggle/input/brain-tumor-progression-tcia/Brain-Tumor-Progression'
out_dir = '/kaggle/working/'

In [2]:
def data_and_mask_to_list(read_dir):
    t1post_images = []
    mask_tumor_image = []
    
    for folder in os.listdir(read_dir):
        if folder.find("T1post") > -1:
            t1post_images_path = [ read_dir + '/' + folder + '/' + image for image in os.listdir(read_dir + '/' + folder) ]
        elif folder.find("MaskTumor") > -1:
            mask_tumor_images_path = [ read_dir + '/' + folder + '/' + image for image in os.listdir(read_dir + '/' + folder) ]

    return list(zip(t1post_images_path, mask_tumor_images_path))

In [3]:
patients = [p for p in os.listdir(input_dir)]
patients.remove('LICENSE')

data_mask_path_list = []

for patient in patients:
    # os.makedirs(out_dir + patient) UNCOMMENT LATER
    for dir in os.listdir(input_dir + '/' + patient):
        data_mask_path_list.extend(data_and_mask_to_list(input_dir + "/" + patient + "/" + dir))

random.shuffle(data_mask_path_list)

train_data_path_list = data_mask_path_list[: int(len(data_mask_path_list)  * 0.8)]
test_data_path_list = data_mask_path_list[int(len(data_mask_path_list) * 0.8):]

trainX, trainY = [x[0] for x in train_data_path_list ], [y[1] for y in train_data_path_list]
# testX, testY = 

In [4]:
def data_path_list(x_path_list, y_path_list):
    print(len(x_path_list) == len(y_path_list))

    # for 

In [5]:
# import torch
# import torchvision

# print("PyTorch version:", torch.__version__)
# print("Torchvision version:", torchvision.__version__)
# print("CUDA is available:", torch.cuda.is_available())

# import sys

# !{sys.executable} -m pip install opencv-python matplotlib
# !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam2.git'

# !mkdir -p images
# !wget -P images https://raw.githubusercontent.com/facebookresearch/sam2/main/notebooks/images/cars.jpg

# !mkdir -p ../checkpoints/
# !wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt

In [6]:
# for IM in image_and_mask:
#     dcm_t1post = pydicom.dcmread(t1post_images_path + '/' + IM[0])
#     dcm_mask = pydicom.dcmread(mask_tumor_image_path + '/' + IM[1])
    
#     img = ds.pixel_array.astype(float)
#     scaled_img = np.uint8((np.maximum(img, 0) / img.max()) * 255)

# cv2.imwrite(write_folder + '/' + image.replace('.dcm','.png'), scaled_img)

In [7]:
!rm -rf /kaggle/working/*